In [1]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('omw-1.4')

stopwords = set(stopwords.words('english'))

import seaborn as sns
import numpy as np

nltk.download('wordnet')
nltk.download('vader_lexicon')

sns.set_theme()

[nltk_data] Downloading package punkt to /Users/vikram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vikram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/vikram/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vikram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/vikram/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
reviews_df = pd.read_csv('/Users/vikram/Documents/COGS108/Group58_SP24/data/fake_reviews.csv')
reviews_df = reviews_df.rename(columns = {'text_': 'text'})
reviews_df['category'] = reviews_df['category'].apply(lambda s: s[:-2].replace('_', ' '))
reviews_df['rating'] = reviews_df['rating'].astype(int)
reviews_df['text_no_punct'] = reviews_df['text'].apply(lambda s: s.lower().translate(str.maketrans('', '', string.punctuation)))
reviews_df.head()

,category,rating,label,text,text_no_punct
0,Home and Kitchen,5,CG,"Love this! Well made, sturdy, and very comfor...",love this well made sturdy and very comfortab...
1,Home and Kitchen,5,CG,"love it, a great upgrade from the original. I...",love it a great upgrade from the original ive...
2,Home and Kitchen,5,CG,This pillow saved my back. I love the look and...,this pillow saved my back i love the look and ...
3,Home and Kitchen,1,CG,"Missing information on how to use it, but it i...",missing information on how to use it but it is...
4,Home and Kitchen,5,CG,Very nice set. Good quality. We have had the s...,very nice set good quality we have had the set...


# Multinomial Naive Bayes Classifier (MNB)

## Bag-Of-Words

### Standard Bag of Words MNB 

In [3]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(reviews_df['text_no_punct'])

X_train, X_test, y_train, y_test = train_test_split(X, reviews_df['label'], test_size=0.2, random_state=42)
clf = MultinomialNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8606405341906764


### Cross-Validation for Bag of Words MNB

In [4]:
# X_train, X_test, y_train, y_test = train_test_split(reviews_df['text_no_punct'], reviews_df['label'], test_size=0.2, random_state=42)

# pipeline_count = Pipeline([
#     ('vectorizer', CountVectorizer()),
#     ('classifier', MultinomialNB())
# ])

# param_grid_count = {
#     'vectorizer__max_features': [None, 5000, 10000],
#     'vectorizer__ngram_range': [(1, 1), (1, 2)],
#     'classifier__alpha': [0.1, 1.0, 10.0]
# }

# grid_search_count = GridSearchCV(pipeline_count, param_grid_count, cv=5, n_jobs=-1, verbose=1)
# grid_search_count.fit(X_train, y_train)

# print(grid_search_count.best_params_)

### Final Bag of Words MNB after Cross-Validation

In [5]:
vectorizer_bow_mnb = {'ngram_range': (1, 2)}
vectorizer = CountVectorizer(**vectorizer_bow_mnb)
X = vectorizer.fit_transform(reviews_df['text_no_punct'])
X_train, X_test, y_train, y_test = train_test_split(X, reviews_df['label'], test_size=0.2, random_state=42)

bow_mnb_params = {'alpha': 0.1,}
mnb_bow = MultinomialNB(**bow_mnb_params)
mnb_bow.fit(X_train, y_train)

y_pred_bow_mnb = mnb_bow.predict(X_test)

accuracy_bow_mnb = accuracy_score(y_test, y_pred_bow_mnb)
print("Accuracy: ", accuracy_bow_mnb)

Accuracy:  0.8908124149870162


## TF-IDF

### Standard TF-IDF MNB

In [6]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(reviews_df['text_no_punct'])

X_train, X_test, y_train, y_test = train_test_split(X, reviews_df['label'], test_size=0.2, random_state=42)

naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

y_pred = naive_bayes.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8712748856188945


### Cross-Validation for TF-IDF MNB

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(reviews_df['text_no_punct'], reviews_df['label'], test_size=0.2, random_state=42)

# pipeline_tfidf = Pipeline([
#     ('tfidf', TfidfVectorizer()),
#     ('classifier', MultinomialNB())
# ])

# param_grid_tfidf = {
#     'tfidf__max_features': [None, 5000, 10000],
#     'tfidf__ngram_range': [(1, 1), (1, 2)],
#     'classifier__alpha': [0.1, 1.0, 10.0]
# }

# grid_search_tfidf = GridSearchCV(pipeline_tfidf, param_grid_tfidf, cv=5, n_jobs=-1, verbose=1)
# grid_search_tfidf.fit(X_train, y_train)

# print(grid_search_tfidf.best_params_)

### Final TF-IDF MNB after Cross-Validation

In [8]:
tfidf_params = {'ngram_range': (1, 2), 'max_features': 10000}
tfidf = TfidfVectorizer(**tfidf_params)
X_tfidf = tfidf.fit_transform(reviews_df['text_no_punct'])
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, reviews_df['label'], test_size=0.2, random_state=42)

mnb_params = {'alpha': 0.1,}
mnb_tfidf = MultinomialNB(**mnb_params)
mnb_tfidf.fit(X_train, y_train)

y_pred_tfidf_mnb = mnb_tfidf.predict(X_test)

accuracy_tfidf_mnb = accuracy_score(y_test, y_pred_tfidf_mnb)
print("Accuracy: ", accuracy_tfidf_mnb)

Accuracy:  0.8993446271794238


# Decision Tree Classifiers

## Bag-of-Words

### Standard BOW Decision Tree Classifier

In [9]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(reviews_df['text_no_punct'])

X_train, X_test, y_train, y_test = train_test_split(X, reviews_df['label'], test_size=0.2, random_state=42)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.7561518486459751


### Cross-Validation for BOW Decision Tree Classifier

In [10]:
# pipeline_count = Pipeline([
#     ('vectorizer', CountVectorizer()),
#     ('classifier', DecisionTreeClassifier(random_state=42))
# ])

# param_grid_count = {
#     'vectorizer__max_features': [None, 5000, 10000],
#     'classifier__max_depth': [None, 10, 20, 30],
#     'classifier__min_samples_split': [2, 5, 10],
#     'classifier__min_samples_leaf': [1, 2, 4]
# }

# grid_search_count = GridSearchCV(pipeline_count, param_grid_count, cv=5)
# grid_search_count.fit(reviews_df['text_no_punct'], reviews_df['label'])
# print(grid_search_count.best_params_)

### Final BOW Decision Tree Classifier after Cross-Validation

In [11]:
best_bow_params = {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 42}

clf_bow = DecisionTreeClassifier(**best_bow_params)
clf_bow.fit(X_train, y_train)

y_pred_bow = clf_bow.predict(X_test)

accuracy_bow_clf = accuracy_score(y_test, y_pred_bow)
print("Accuracy:", accuracy_bow_clf)

Accuracy: 0.7597378508717695


## TF-IDF

### Standard TF-IDF Decision Tree Classifier

In [12]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(reviews_df['text_no_punct'])

X_train, X_test, y_train, y_test = train_test_split(X, reviews_df['label'], test_size=0.2, random_state=42)

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7797700012365525


### Cross-Validation for TF-IDF Decision Tree Classifier

In [13]:
# pipeline_tfidf = Pipeline([
#     ('vectorizer', TfidfVectorizer()),
#     ('classifier', DecisionTreeClassifier(random_state=42))
# ])

# param_grid_tfidf = {
#     'vectorizer__max_features': [None, 5000, 10000],
#     'classifier__max_depth': [None, 10, 20, 30],
#     'classifier__min_samples_split': [2, 5, 10],
#     'classifier__min_samples_leaf': [1, 2, 4]

# grid_search_tfidf = GridSearchCV(pipeline_tfidf, param_grid_tfidf, cv=5)
# grid_search_tfidf.fit(reviews_df['text_no_punct'], reviews_df['label'])
# print(grid_search_tfidf.best_params_)
# # }

### Final TF-IDF Decision Tree Classifier after Cross-Validation

In [14]:
best_tfidf_params = {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 42}

clf_tfidf = DecisionTreeClassifier(**best_tfidf_params)
clf_tfidf.fit(X_train, y_train)

y_pred_tfidf = clf_tfidf.predict(X_test)

accuracy_tfidf_clf = accuracy_score(y_test, y_pred_tfidf)
print("Accuracy:", accuracy_tfidf_clf)


Accuracy: 0.787931247681464


In [15]:
print("MultinomialNB Bag-Of-Words Accuracy: ", accuracy_bow_mnb)
print("MultinomialNB TF-IDF Accuracy: ", accuracy_tfidf_mnb)
print("DecisionTreeClassifier Bag-Of-Words Accuracy: ", accuracy_bow_clf)
print("DecisionTreeClassifier TF-IDF Accuracy: ", accuracy_tfidf_clf)


MultinomialNB Bag-Of-Words Accuracy:  0.8908124149870162
MultinomialNB TF-IDF Accuracy:  0.8993446271794238
DecisionTreeClassifier Bag-Of-Words Accuracy:  0.7597378508717695
DecisionTreeClassifier TF-IDF Accuracy:  0.787931247681464


In [16]:
y_pred_bow_mnb

array(['CG', 'CG', 'OR', ..., 'CG', 'OR', 'OR'], dtype='<U2')

In [17]:
y_pred_tfidf_mnb

array(['CG', 'CG', 'OR', ..., 'CG', 'OR', 'OR'], dtype='<U2')

In [22]:
res = pd.read_csv('results.csv')
res['MNB_BOW'] = y_pred_bow_mnb
res['MNB_TF-IDF'] = y_pred_tfidf_mnb
res.to_csv('results.csv')